In [331]:
file = open('input.txt', 'r')
input = file.readlines()
file.close()

In [332]:
#dict with name as key
modules = {}
#queue with tuples (source_module_name, dest_module_name, pulse (True/False))
pulse_queue = []

In [333]:
class FlipFlop:
    def __init__(self, name, outputs) -> None:
        self.name = name
        self.state_on = False
        self.output_modules = outputs
        pass
    
    def set_pulse(self, pulse, input_module = None):
        if pulse:
            return
        else:
            self.state_on = not self.state_on
            #send pulse according to state
            for module in self.output_modules:
                pulse_queue.append((self.name, module, self.state_on))
            return
    
    def to_string(self):
        return f"FlipFlop: {self.name}, output modules: {self.output_modules}, state "+("on" if self.state_on else "off")

In [334]:
class Conjunction:
    def __init__(self, name, outputs) -> None:
        self.name = name
        self.connected_inputs = {}
        self.output_modules = outputs
        pass

    def set_input_module(self, input_module):
        # False -> low, True -> high
        self.connected_inputs[input_module] = False

    def set_pulse(self, pulse, input_module):
        self.connected_inputs[input_module] = pulse
        send = True
        if all(value == True for value in self.connected_inputs.values()):
            send = False
        for module in self.output_modules:
            pulse_queue.append((self.name, module, send))
        return
    
    def to_string(self):
        return f"Conjunction: {self.name}, input modules: {self.connected_inputs}, output modules: {self.output_modules}"

In [335]:
class Broadcast:
    def __init__(self, name, outputs) -> None:
        self.name = name
        self.output_modules = outputs
        pass

    def set_pulse(self, pulse, input_module = None):
        for module in self.output_modules:
            pulse_queue.append((self.name, module, pulse))
        return
    
    def to_string(self):
        return f"Broadcast: {self.name}, output modules: {self.output_modules}"

In [336]:
for line in input:
    print(line.strip())
    linesplit = line.split()
    outputs = line.strip().split('->')[1].split(',')
    outputs = [s.strip() for s in outputs]
    if linesplit[0] == 'broadcaster':
        modules[linesplit[0]] = Broadcast(linesplit[0], outputs)
    elif linesplit[0][0] == '%':
        modules[linesplit[0][1:]] = FlipFlop(linesplit[0][1:],outputs)
    elif linesplit[0][0] == '&':
        modules[linesplit[0][1:]] = Conjunction(linesplit[0][1:],outputs)

#find input modules for Conjunctions
for module in modules.values():
    if isinstance(module, Conjunction):
        for input in modules.values():
            if module.name in input.output_modules:
                module.set_input_module(input.name)

print()
for module in modules.values():
    print(module.to_string())

%rp -> gq, sd
&kh -> cs
%jz -> pl, jb
%dx -> tx
%dh -> bm, sd
&zv -> ns, dx, hl, hn, fm
%xb -> ds, sk
%hv -> sk, kr
%db -> zv, zz
&sk -> rg, hh, hv, kr, kh, zl, zn
%tc -> jz
%dj -> ts, pl
%jk -> sd, vh
%fm -> dx, zv
%dp -> sd, cc
%vh -> sd
&lz -> cs
%kr -> rg
%jb -> bf, pl
%kz -> sk
%ts -> pl, bs
%gr -> ns, zv
%kc -> sd, kf
%jd -> zv
%bs -> vg
%zk -> rp
%vf -> zk
%mm -> ms, sk
%qc -> pl, dj
%fk -> qc
%bm -> vf, sd
%ds -> kz, sk
%sn -> zv, jd
%zn -> mm
%ct -> fk
%np -> sk, xb
&tg -> cs
%tx -> cm, zv
%zl -> hh
%zz -> px, zv
%ms -> zl, sk
%ns -> db
%px -> zv, sn
broadcaster -> fm, hv, kc, bv
&hn -> cs
%hh -> np
%kf -> dh
%vg -> pl, tc
%bv -> ct, pl
&pl -> bv, fk, ct, bs, tg, tc
%cm -> zv, hl
%cc -> sd, jk
%bf -> pl
%hl -> gr
&cs -> rx
%gq -> dp
%rg -> zn
&sd -> zk, kf, gq, lz, kc, vf

FlipFlop: rp, output modules: ['gq', 'sd'], state off
Conjunction: kh, input modules: {'sk': False}, output modules: ['cs']
FlipFlop: jz, output modules: ['pl', 'jb'], state off
FlipFlop: dx, output modules:

In [337]:
#press button i times and work queue
high_pulses = 0
low_pulses = 0
for i in range(1000):
    pulse_queue.append(('start','broadcaster',False))
    while (len(pulse_queue) > 0):
        next_pulse = pulse_queue.pop(0)
        #print(next_pulse)
        if (next_pulse[2]):
            high_pulses += 1
        else:
            low_pulses += 1
        if (next_pulse[1] not in modules):
            continue
        else:
            modules[next_pulse[1]].set_pulse(next_pulse[2], next_pulse[0])
        #print(pulse_queue)
    

In [338]:
print(f"low: {low_pulses}, high: {high_pulses}")
print(low_pulses*high_pulses)

low: 16669, high: 42193
703315117
